In [1]:
#!pip install scikit-image

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import img_as_ubyte,measure,color,morphology, io, exposure
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, auc, confusion_matrix, precision_score, pairwise
from imblearn.over_sampling import RandomOverSampler
import os


In [3]:
#Resize all images to 768x1024 and output to interim
for filename in os.listdir("../data/ISIC_2017/ISIC-2017_Training_Part1_GroundTruth"):
    if 'DS' in filename: 
        continue
    temp = plt.imread("../data/ISIC_2017/ISIC-2017_Training_Part1_GroundTruth/"+filename)
    temp_resized = resize(temp, (768, 1024),anti_aliasing=True)

    new_filename = filename[:5]+"re_"+filename[5:]                 
    io.imsave('../data/ISIC_2017/segmentations_re/'+new_filename, img_as_ubyte(temp_resized), check_contrast = False)

KeyboardInterrupt: 

In [ ]:
#Create custom masks for all images (without using preexisting masks)
#If it cannot create a better mask, then default to the provided mask
images = os.listdir('../data/ISIC_2017/Images_re')

#greyscaling function
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def create_masks():
    for filename in['ISIC_re_0008552.jpg']: #for images in (folder with original images)
        temp = plt.imread('../data/ISIC_2017/Images_re/'+filename) #Import original image
        temp_seg = plt.imread('../data/ISIC_2017/segmentations_re/'+filename[:-4]+"_segmentation.png") #Import segmentation of that image

        #Slightly increase contrast for better color detection 
        temp = exposure.rescale_intensity(temp) 

        #Keep green values, blue values and greyscale images in an attempt
        #to make red colors as low as possible (since most of the lesions are red)
        gray_1 = exposure.rescale_intensity(temp[:,:,1]) #green only
        gray_2 = exposure.rescale_intensity(temp[:,:,2]) #blue only
        gray_3 = exposure.rescale_intensity(rgb2gray(temp)) #all greyscaled

        imlist = []
        difflist = []
        same_counter = 0
        index = 0
        for quant in np.arange(0.15,1,0.075): #Arbitrary quantiles for color histogram

            #first mask with green only
            mymask_1 = gray_1 < np.quantile(gray_1,quant) 
    
            #second mask with blue only
            mymask_2 = gray_2 < np.quantile(gray_2,quant) 
           
            #third mask with greyscaled image
            mymask_3 = gray_3 < np.quantile(gray_3,quant)
            
            #Add all masks and keep joint elements
            better_mask = mymask_1.astype('int32')+mymask_2.astype('int32')+mymask_3.astype('int32')
            fixed_mask = better_mask.copy() + temp_seg
            
            #make mask binary (or rather 0's and 1's) then make morphology corrections
            fixed_mask[fixed_mask < 4] = 0
            fixed_mask[fixed_mask == 4] = 1
            fixed_mask = fixed_mask.astype("bool")
            #Morphology transformations to generalize mask
            fixed_mask = morphology.binary_opening(fixed_mask,morphology.disk(2)) #removes hairs
            fixed_mask = morphology.binary_closing(fixed_mask,morphology.disk(3)) #Closes small disconnected areas
            #fixed_mask = morphology.binary_dilation(fixed_mask,morphology.disk(2)) #makes mask slightly bigger as it is usually a bit too small

            fixed_mask = morphology.remove_small_holes(fixed_mask,100000) #Fills in holes and gaps in the mask
            fixed_mask = morphology.remove_small_objects(fixed_mask,3000) #Removes small clusters outside the main masked area

            #calculate size difference between current mask and original segmentation
            diff = np.abs((np.sum(fixed_mask))/np.sum(temp_seg)) 

            #keep the fixed_mask if size is at least 5% different than the original segmentation
            #mask cannot take 100% of the image size (768x1024)
            if((diff > 1.05 or diff < 0.95) and np.sum(fixed_mask)!=786432 and np.sum(fixed_mask!=0)) :
                imlist.append(fixed_mask)
                difflist.append(np.abs(1-diff))
                index+=1
                same_counter = 0
            else:
                same_counter += 1


            #break when best mask has been found
            if index >=2:
                if difflist[index-1] > difflist[index-2] or same_counter >= 3:
                    break



        #print image
        mask = imlist[difflist.index(min(difflist))] if imlist else temp_seg
        new_filename = filename[:-4]+"_mask.jpg"                
        io.imsave('../data/ISIC_2017/custom_masks/'+new_filename, img_as_ubyte(mask), check_contrast = False)

In [ ]:
for filename in os.listdir("../data/raw/ISIC_2017/Images/ISIC-2017_Training_Data"):
    if "superpixels" in filename:
        os.remove('../data/raw/ISIC_2017/Images/ISIC-2017_Training_Data/'+filename)

In [ ]:
create_masks()